# Mindcon 食物分类和西安旅游分类通用notebook

## 1.下载数据到notebook环境

In [3]:
import moxing as mox
mox.file.copy_parallel('obs://mindcon00001/raw/train_data', '/home/ma-user/work/raw')
mox.file.copy_parallel('obs://mindcon00001/mindcon_xian_travel/test', '/home/ma-user/work/test')
mox.file.copy('obs://mindcon00001/raw/label_id_name.json','/home/ma-user/work/label_id_name.json')


INFO:root:Listing OBS: 1000
INFO:root:Listing OBS: 2000
INFO:root:Listing OBS: 3000
INFO:root:Listing OBS: 4000
INFO:root:Listing OBS: 5000
INFO:root:Listing OBS: 6000
INFO:root:Listing OBS: 7000
INFO:root:Listing OBS: 1000
INFO:root:Listing OBS: 2000
INFO:root:Listing OBS: 3000
INFO:root:Listing OBS: 4000
INFO:root:Listing OBS: 5000
INFO:root:Listing OBS: 6000
INFO:root:Listing OBS: 7000
INFO:root:pid: None.	1000/7462
INFO:root:pid: None.	2000/7462
INFO:root:pid: None.	3000/7462
INFO:root:pid: None.	4000/7462
INFO:root:pid: None.	5000/7462
INFO:root:pid: None.	6000/7462
INFO:root:pid: None.	7000/7462


## 2.打开终端，下载预训练模型
```
mkdir ckpt
cd ckpt
wget https://download.mindspore.cn/vision/classification/vit_b_16_224.ckpt
pwd
```
## 3.处理数据方便MindSpore读取。

处理成标签为文件夹，每个文件夹内含有对应图片


In [2]:
import os
import shutil

In [3]:
path = './raw/'
target_path = './data/train/'

files = os.listdir(path)
for file in files:
    if file.endswith('.txt'):
        with open(path + file,'r') as f:
            line = f.read()
            image, label = line.split(', ')
            if not os.path.exists(target_path + label):
                os.makedirs(target_path + label)
            shutil.copyfile(path + image, target_path + label + '/' + image)

## 4.数据增强，主要参考了MindsSpore的[mindspore.dataset.vision](https://mindspore.cn/docs/zh-CN/r1.9/api_python/mindspore.dataset.vision.html) API

其中变换方式添加了随机翻转，随机亮度对比度

随机翻转从0.5提升到0.75的时候，验证集精度能提高1%

In [9]:
import mindspore.dataset as ds
import mindspore.dataset.vision.c_transforms as vision

def get_dataset():
    image_folder_dataset_dir = "./data/train"
    mapping = {}
    for i in range(54):
        mapping[str(i)] = int(i)

    mean = [0.485 * 255, 0.456 * 255, 0.406 * 255]
    std = [0.229 * 255, 0.224 * 255, 0.225 * 255]

    transforms_list = [vision.RandomCropDecodeResize(size=224,
                                                    scale=(0.08, 1.0),
                                                    ratio=(0.75, 1.333)),
                    vision.RandomHorizontalFlip(0.8),
                    vision.RandomVerticalFlip(0.8),
                    vision.RandomColorAdjust(brightness=(0.5, 1.25),contrast=(0.4, 1.1),saturation=(0.3, 1)),
                    vision.Normalize(mean=mean, std=std),
                    vision.HWC2CHW(),
                    ]

    dataset = ds.ImageFolderDataset(dataset_dir=image_folder_dataset_dir,
                                    shuffle=True,
                                    num_parallel_workers=8,
                                    class_indexing=mapping
                                    )

    dataset = dataset.map(operations=transforms_list)

    dataset = dataset.batch(32, drop_remainder=True, num_parallel_workers=8)

    train_dataset, val_dataset = dataset.split([0.9, 0.1])
    
    return train_dataset, val_dataset

建立网络，参考官方的ViT模型：
[Vision Transformer图像分类](https://mindspore.cn/tutorials/application/zh-CN/r1.9/cv/vit.html)

In [10]:
from mindspore import nn, ops
import mindspore as ms
from typing import Optional
from mindspore.common.initializer import Normal
from mindspore.common.initializer import initializer
from mindspore import Parameter

def init(init_type, shape, dtype, name, requires_grad):
    """Init."""
    initial = initializer(init_type, shape, dtype).init_data()
    return Parameter(initial, name=name, requires_grad=requires_grad)

class ViT(nn.Cell):
    def __init__(self,
                 image_size: int = 224,
                 input_channels: int = 3,
                 patch_size: int = 16,
                 embed_dim: int = 768,
                 num_layers: int = 12,
                 num_heads: int = 12,
                 mlp_dim: int = 3072,
                 keep_prob: float = 1.0,
                 attention_keep_prob: float = 1.0,
                 drop_path_keep_prob: float = 1.0,
                 activation: nn.Cell = nn.GELU,
                 norm: Optional[nn.Cell] = nn.LayerNorm,
                 pool: str = 'cls',
                 num_classes=1000) -> None:
        super(ViT, self).__init__()

        self.patch_embedding = PatchEmbedding(image_size=image_size,
                                              patch_size=patch_size,
                                              embed_dim=embed_dim,
                                              input_channels=input_channels)
        num_patches = self.patch_embedding.num_patches

        # 此处增加class_embedding和pos_embedding，如果不是进行分类任务
        # 可以只增加pos_embedding，通过pool参数进行控制
        self.cls_token = init(init_type=Normal(sigma=1.0),
                              shape=(1, 1, embed_dim),
                              dtype=ms.float32,
                              name='cls',
                              requires_grad=True)

        # pos_embedding也是一组可以学习的参数，会被加入到经过处理的patch矩阵中
        self.pos_embedding = init(init_type=Normal(sigma=1.0),
                                  shape=(1, num_patches + 1, embed_dim),
                                  dtype=ms.float32,
                                  name='pos_embedding',
                                  requires_grad=True)

        # axis=1定义了会在向量的开头加入class_embedding
        self.concat = ops.Concat(axis=1)

        self.pool = pool
        self.pos_dropout = nn.Dropout(keep_prob)
        self.norm = norm((embed_dim,))
        self.tile = ops.Tile()
        self.transformer = TransformerEncoder(dim=embed_dim,
                                              num_layers=num_layers,
                                              num_heads=num_heads,
                                              mlp_dim=mlp_dim,
                                              keep_prob=keep_prob,
                                              attention_keep_prob=attention_keep_prob,
                                              drop_path_keep_prob=drop_path_keep_prob,
                                              activation=activation,
                                              norm=norm)
        self.dropout = nn.Dropout(keep_prob)
        self.dense = nn.Dense(embed_dim, num_classes)

    def construct(self, x):
        """ViT construct."""
        x = self.patch_embedding(x)

        # class_embedding主要借鉴了BERT模型的用于文本分类时的思想
        # 在每一个word vector之前增加一个类别值，通常是加在向量的第一位
        cls_tokens = self.tile(self.cls_token, (x.shape[0], 1, 1))
        x = self.concat((cls_tokens, x))
        x += self.pos_embedding

        x = self.pos_dropout(x)
        x = self.transformer(x)
        x = self.norm(x)

        # 增加的class_embedding是一个可以学习的参数，经过网络的不断训练
        # 最终以输出向量的第一个维度的输出来决定最后的输出类别；
        x = x[:, 0]

        if self.training:
            x = self.dropout(x)
        x = self.dense(x)
        return x

class Attention(nn.Cell):
    def __init__(self,
                 dim: int,
                 num_heads: int = 8,
                 keep_prob: float = 1.0,
                 attention_keep_prob: float = 1.0):
        super(Attention, self).__init__()

        self.num_heads = num_heads
        head_dim = dim // num_heads
        self.scale = ms.Tensor(head_dim ** -0.5)

        self.qkv = nn.Dense(dim, dim * 3)
        self.attn_drop = nn.Dropout(attention_keep_prob)
        self.out = nn.Dense(dim, dim)
        self.out_drop = nn.Dropout(keep_prob)

        self.mul = ops.Mul()
        self.reshape = ops.Reshape()
        self.transpose = ops.Transpose()
        self.unstack = ops.Unstack(axis=0)
        self.attn_matmul_v = ops.BatchMatMul()
        self.q_matmul_k = ops.BatchMatMul(transpose_b=True)
        self.softmax = nn.Softmax(axis=-1)

    def construct(self, x):
        """Attention construct."""
        b, n, c = x.shape

        # 最初的输入向量首先会经过Embedding层映射成Q(Query)，K(Key)，V(Value)三个向量
        # 由于是并行操作，所以代码中是映射成为dim*3的向量然后进行分割
        qkv = self.qkv(x)

        #多头注意力机制就是将原本self-Attention处理的向量分割为多个Head进行处理
        qkv = self.reshape(qkv, (b, n, 3, self.num_heads, c // self.num_heads))
        qkv = self.transpose(qkv, (2, 0, 3, 1, 4))
        q, k, v = self.unstack(qkv)

        # 自注意力机制的自注意主要体现在它的Q，K，V都来源于其自身
        # 也就是该过程是在提取输入的不同顺序的向量的联系与特征
        # 最终通过不同顺序向量之间的联系紧密性（Q与K乘积经过Softmax的结果）来表现出来
        attn = self.q_matmul_k(q, k)
        attn = self.mul(attn, self.scale)
        attn = self.softmax(attn)
        attn = self.attn_drop(attn)

        # 其最终输出则是通过V这个映射后的向量与QK经过Softmax结果进行weight sum获得
        # 这个过程可以理解为在全局上进行自注意表示
        out = self.attn_matmul_v(attn, v)
        out = self.transpose(out, (0, 2, 1, 3))
        out = self.reshape(out, (b, n, c))
        out = self.out(out)
        out = self.out_drop(out)
        

        return out

class FeedForward(nn.Cell):
    def __init__(self,
                 in_features: int,
                 hidden_features: Optional[int] = None,
                 out_features: Optional[int] = None,
                 activation: nn.Cell = nn.GELU,
                 keep_prob: float = 1.0):
        super(FeedForward, self).__init__()
        out_features = out_features or in_features
        hidden_features = hidden_features or in_features
        self.dense1 = nn.Dense(in_features, hidden_features)
        self.activation = activation()
        self.dense2 = nn.Dense(hidden_features, out_features)
        self.dropout = nn.Dropout(keep_prob)

    def construct(self, x):
        """Feed Forward construct."""
        x = self.dense1(x)
        x = self.activation(x)
        x = self.dropout(x)
        x = self.dense2(x)
        x = self.dropout(x)

        return x


class ResidualCell(nn.Cell):
    def __init__(self, cell):
        super(ResidualCell, self).__init__()
        self.cell = cell

    def construct(self, x):
        """ResidualCell construct."""
        return self.cell(x) + x


class TransformerEncoder(nn.Cell):
    def __init__(self,
                 dim: int,
                 num_layers: int,
                 num_heads: int,
                 mlp_dim: int,
                 keep_prob: float = 1.,
                 attention_keep_prob: float = 1.0,
                 drop_path_keep_prob: float = 1.0,
                 activation: nn.Cell = nn.GELU,
                 norm: nn.Cell = nn.LayerNorm):
        super(TransformerEncoder, self).__init__()
        layers = []

        # 从vit_architecture图可以发现，多个子encoder的堆叠就完成了模型编码器的构建
        # 在ViT模型中，依然沿用这个思路，通过配置超参数num_layers，就可以确定堆叠层数
        for _ in range(num_layers):
            normalization1 = norm((dim,))
            normalization2 = norm((dim,))
            attention = Attention(dim=dim,
                                  num_heads=num_heads,
                                  keep_prob=keep_prob,
                                  attention_keep_prob=attention_keep_prob)

            feedforward = FeedForward(in_features=dim,
                                      hidden_features=mlp_dim,
                                      activation=activation,
                                      keep_prob=keep_prob)

            # ViT模型中的基础结构与标准Transformer有所不同
            # 主要在于Normalization的位置是放在Self-Attention和Feed Forward之前
            # 其他结构如Residual Connection，Feed Forward，Normalization都如Transformer中所设计
            layers.append(
                nn.SequentialCell([
                    # Residual Connection，Normalization的结构可以保证模型有很强的扩展性
                    # 保证信息经过深层处理不会出现退化的现象，这是Residual Connection的作用
                    # Normalization和dropout的应用可以增强模型泛化能力
                    ResidualCell(nn.SequentialCell([normalization1,
                                                    attention])),

                    ResidualCell(nn.SequentialCell([normalization2,
                                                    feedforward]))
                ])
            )
        self.layers = nn.SequentialCell(layers)

    def construct(self, x):
        """Transformer construct."""
        return self.layers(x)


class PatchEmbedding(nn.Cell):
    MIN_NUM_PATCHES = 4
    def __init__(self,
                 image_size: int = 224,
                 patch_size: int = 16,
                 embed_dim: int = 768,
                 input_channels: int = 3):
        super(PatchEmbedding, self).__init__()

        self.image_size = image_size
        self.patch_size = patch_size
        self.num_patches = (image_size // patch_size) ** 2

        # 通过将输入图像在每个channel上划分为16*16个patch
        self.conv = nn.Conv2d(input_channels, embed_dim, kernel_size=patch_size, stride=patch_size, has_bias=True)
        self.reshape = ops.Reshape()
        self.transpose = ops.Transpose()

    def construct(self, x):
        """Path Embedding construct."""
        x = self.conv(x)
        b, c, h, w = x.shape

        # 再将每一个patch的矩阵拉伸成为一个1维向量，从而获得了近似词向量堆叠的效果；
        x = self.reshape(x, (b, c, h * w))
        x = self.transpose(x, (0, 2, 1))

        return x

定义损失函数

In [11]:
from mindspore.nn import LossBase
from mindspore import nn, ops
import mindspore as ms
from mindspore.common.initializer import One

class CrossEntropySmooth(LossBase):
    """CrossEntropy."""

    def __init__(self, sparse=True, reduction='mean', smooth_factor=0., num_classes=1000):
        super(CrossEntropySmooth, self).__init__()
        self.onehot = ops.OneHot()
        self.sparse = sparse
        self.on_value = ms.Tensor(1.0 - smooth_factor, ms.float32)
        self.off_value = ms.Tensor(1.0 * smooth_factor / (num_classes - 1), ms.float32)
        self.ce = nn.SoftmaxCrossEntropyWithLogits(reduction=reduction)

    def construct(self, logit, label):
        if self.sparse:
            label = self.onehot(label, ops.shape(logit)[1], self.on_value, self.off_value)
        loss = self.ce(logit, label)
        return loss

logits = ms.Tensor(shape = (32, 54), dtype=ms.float32, init=One())
label = ms.Tensor(shape = (32,), dtype=ms.int32, init=One())
network_loss = CrossEntropySmooth(sparse=True,
                                  reduction="mean",
                                  smooth_factor=0.1,
                                  num_classes=54)
loss = network_loss(logits, label)

定义网络

In [ ]:
import mindspore as ms
# from vit import ViT
from mindspore.train.callback import LossMonitor, TimeMonitor, CheckpointConfig, ModelCheckpoint
from mindspore import nn, context
# from mindvision.engine.callback import ValAccMonitor


class CustomWithLossCell(nn.Cell):
    """连接前向网络和损失函数"""

    def __init__(self, backbone, loss_fn):
        """前向网络backbone和损失函数loss_fn"""
        super(CustomWithLossCell, self).__init__(auto_prefix=False)
        self._backbone = backbone
        self._loss_fn = loss_fn

    def construct(self, data, label):
        output = self._backbone(data)                 # 前向计算得到网络输出
        return self._loss_fn(output, label)  # 得到多标签损失值

class CustomWithEvalCell(nn.Cell):
    """自定义多标签评估网络"""

    def __init__(self, network):
        super(CustomWithEvalCell, self).__init__(auto_prefix=False)
        self.network = network

    def construct(self, data, label):
        output = self.network(data)
        return output, label


开示训练

In [13]:


ms.set_context(mode=ms.GRAPH_MODE,device_target="Ascend")
train_data, val_data = get_dataset()

network = ViT()
vit_path = "./ckpt/vit_b_16_224.ckpt"
param_dict = ms.load_checkpoint(vit_path)
ms.load_param_into_net(network, param_dict)
network.dense = nn.Dense(768, 54)

epoch_size = 40
momentum = 0.9
num_classes = 54
step_size = train_data.get_dataset_size()

lr = nn.cosine_decay_lr(min_lr=float(0),
                        max_lr=0.00005,
                        total_step=epoch_size * step_size,
                        step_per_epoch=step_size,
                        decay_epoch=10)
network_opt = nn.Adam(network.trainable_params(), lr, momentum)
network_loss = CrossEntropySmooth(sparse=True,
                                    reduction="mean",
                                    smooth_factor=0.1,
                                    num_classes=num_classes)

loss_net = CustomWithLossCell(network, network_loss)
eval_net = CustomWithEvalCell(network)
    
ckpt_config = CheckpointConfig(save_checkpoint_steps=3 * step_size, keep_checkpoint_max=100)
ckpt_callback = ModelCheckpoint(prefix='vit_b_16', directory='./ViT1', config=ckpt_config)
ascend_target = (ms.get_context("device_target") == "Ascend")
if ascend_target:
    model = ms.Model(loss_net, optimizer=network_opt, eval_network=eval_net, metrics={"acc"}, amp_level="O2")
else:
    model = ms.Model(loss_net, optimizer=network_opt, eval_network=eval_net, metrics={"acc"}, amp_level="O0")
model.train(epoch_size,
            train_data,
            callbacks=[ckpt_callback, LossMonitor(step_size), TimeMonitor(step_size)]
            )
result = model.eval(val_data)
print(result)

[WARNING] ME(6018:281473032386432,MainProcess):2023-01-15-07:43:09.298.362 [mindspore/dataset/engine/datasets.py:1122] Dataset is shuffled before split.
[WARNING] DEVICE(6018,ffff8c1bf780,python):2023-01-15-07:43:38.895.369 [mindspore/ccsrc/plugin/device/ascend/hal/device/kernel_select_ascend.cc:330] FilterRaisedOrReducePrecisionMatchedKernelInfo] Operator:[StridedSliceGrad] don't support int64, reduce precision from int64 to int32.


epoch: 1 step: 104, loss is 2.5859375
epoch time: 74268.370 ms, per step time: 714.119 ms
epoch: 2 step: 104, loss is 1.4765625
epoch time: 10257.620 ms, per step time: 98.631 ms
epoch: 3 step: 104, loss is 1.1494140625
epoch time: 14763.643 ms, per step time: 141.958 ms
epoch: 4 step: 104, loss is 1.0556640625
epoch time: 10262.417 ms, per step time: 98.677 ms
epoch: 5 step: 104, loss is 0.94287109375
epoch time: 10266.959 ms, per step time: 98.721 ms
epoch: 6 step: 104, loss is 0.974609375
epoch time: 14379.265 ms, per step time: 138.262 ms
epoch: 7 step: 104, loss is 0.7646484375
epoch time: 10269.441 ms, per step time: 98.745 ms
epoch: 8 step: 104, loss is 0.8125
epoch time: 10269.266 ms, per step time: 98.743 ms
epoch: 9 step: 104, loss is 0.81787109375
epoch time: 14172.830 ms, per step time: 136.277 ms
epoch: 10 step: 104, loss is 0.79833984375
epoch time: 10257.586 ms, per step time: 98.631 ms
epoch: 11 step: 104, loss is 0.78466796875
epoch time: 10252.382 ms, per step time: 9

In [14]:
result = model.eval(val_data)
print(result)

[WARNING] MD(6018,ffff8c1bf780,python):2023-01-15-07:54:32.340.027 [mindspore/ccsrc/minddata/dataset/engine/datasetops/device_queue_op.cc:73] ~DeviceQueueOp] preprocess_batch: 12; batch_queue: 7, 7, 6, 6, 5, 5, 4, 4, 3, 3; push_start_time: 2023-01-15-07:52:16.988.667, 2023-01-15-07:52:17.007.056, 2023-01-15-07:52:17.041.176, 2023-01-15-07:52:17.057.230, 2023-01-15-07:52:17.061.301, 2023-01-15-07:52:17.066.119, 2023-01-15-07:52:17.070.183, 2023-01-15-07:52:17.079.712, 2023-01-15-07:52:17.098.703, 2023-01-15-07:52:17.129.798; push_end_time: 2023-01-15-07:52:16.993.428, 2023-01-15-07:52:17.011.065, 2023-01-15-07:52:17.045.322, 2023-01-15-07:52:17.061.259, 2023-01-15-07:52:17.066.076, 2023-01-15-07:52:17.070.136, 2023-01-15-07:52:17.074.622, 2023-01-15-07:52:17.083.757, 2023-01-15-07:52:17.102.834, 2023-01-15-07:52:17.134.283.


{'acc': 0.9869791666666666}


In [ ]:
from PIL import Image
import os
import numpy as np
import json

import mindspore as ms
from mindspore import Tensor

# from vit import ViT

def process_image(path):

    image_list = []

    mean = np.array([0.485 * 255, 0.456 * 255, 0.406 * 255])
    std = np.array([0.229 * 255, 0.224 * 255, 0.225 * 255])

    files = os.listdir(path)
    files = sorted(files, key=lambda x:int(x.split('.')[0]))
    
    for file in files:
        image = Image.open(path + file).convert("RGB")
        image = image.resize((224, 224))
        # plt.imshow(image)
        image = (image - mean) / std
        image = image.astype(np.float32)
        image = np.transpose(image, (2, 0, 1))
        image = np.expand_dims(image, axis=0)
        image_list.append(image)
    return image_list




ms.set_context(device_target="CPU")
# ms.set_context(device_target="Ascend")
images = process_image("./test/")

network = ViT(num_classes=54)
vit_path = "./ViT1/vit_b_16-40_104.ckpt"
param_dict = ms.load_checkpoint(vit_path)
ms.load_param_into_net(network, param_dict)

model = ms.Model(network)

results = []
for image in images:
    pre = model.predict(Tensor(image,ms.float32))
    result = np.argmax(pre)
    results.append(result)
    print(result)
        
with open("result0114.txt","w") as f:
    for result in results:
        f.write(str(result))
        f.write('\n')
print(results)


In [2]:
import moxing as mox
mox.file.copy('./ViT1/vit_b_16-40_104.ckpt','obs://mindcon00001/xian/vit_b_16-40_104.ckpt')

